In [ ]:
!pip install langchain-community==0.2.15 langchain-chroma==0.1.3 langchain-text-splitters==0.2.2 langchain-huggingface==0.0.3 langchain-groq==0.1.9 unstructured==0.15.0 unstructured[pdf]==0.15.0 nltk==3.8.1

In [ ]:
!apt-get install poppler-utils
!apt install tesseract-ocr

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 49 not upgraded.
Need to get 186 kB of archives.
After this operation, 696 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.5 [186 kB]
Fetched 186 kB in 1s (203 kB/s)
Selecting previously unselected package poppler-utils.
(Reading database ... 123597 files and directories currently installed.)
Preparing to unpack .../poppler-utils_22.02.0-2ubuntu0.5_amd64.deb ...
Unpacking poppler-utils (22.02.0-2ubuntu0.5) ...
Setting up poppler-utils (22.02.0-2ubuntu0.5) ...
Processing triggers for man-db (2.10.2-1) ...
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
T

In [ ]:
import os

from langchain_community.document_loaders import UnstructuredPDFLoader, DirectoryLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma
from langchain_groq import ChatGroq
from langchain.chains import RetrievalQA

In [ ]:
from google.colab import userdata
GROQ_API_KEY = userdata.get('GROQ_API_KEY')

In [ ]:
from google.colab import userdata
OPENAI_KEY=userdata.get('OPENAI_KEY')

In [ ]:
#GROQ_API_KEY = "your_groq_api_key"

In [ ]:
os.environ["GROQ_API_KEY"] = GROQ_API_KEY

In [ ]:
link="https://drive.google.com/drive/u/0/folders/1dm5XU24B3xV1zzM7x_vKPOYfBMUgX9zp"

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import glob
pdf_files = glob.glob("/content/drive/MyDrive/multi doc qa rag/med folder/*.pdf")

In [ ]:
documents=pdf_files

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
pip install pymupdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 40.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 89.8 MB/s eta 0:00:00


In [ ]:
from langchain.document_loaders import DirectoryLoader
from langchain.document_loaders.pdf import PyMuPDFLoader

# Path to your PDFs in the subfolder "google colabs" of "My Drive"
pdf_path = r'/content/drive/My Drive/Colab Notebooks/multidocqarag/medfolder'

# Load the PDFs from the specified directory
loader = DirectoryLoader(
    path=pdf_path,
    glob="**/*.pdf",  # This will match all PDF files
    loader_cls=PyMuPDFLoader
)
documents = loader.load()

print(f"Loaded {len(documents)} documents")


Loaded 28 documents


In [ ]:
pip install PyDrive


In [ ]:
text_splitter = CharacterTextSplitter(
    chunk_size=2000,
    chunk_overlap=500
)

text_chunks = text_splitter.split_documents(documents)

In [ ]:
persist_directory = "doc_db"

In [ ]:
embedding = HuggingFaceEmbeddings()

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
pip install openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.1/374.1 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 15.8 MB/s eta 0:00:00


In [ ]:
from langchain.embeddings import OpenAIEmbeddings

# Initialize the embedding model
embedding = OpenAIEmbeddings(openai_api_key=OPENAI_KEY)


In [ ]:
pip install tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 21.3 MB/s eta 0:00:00


In [ ]:
vectorstore = Chroma.from_documents(
    documents=text_chunks,
    embedding=embedding,
    persist_directory=persist_directory
)

In [ ]:
retriever = vectorstore.as_retriever()

In [ ]:
groq_api_key=GROQ_API_KEY


In [ ]:
llm = ChatGroq(
    model="llama-3.1-8b-instant",
    temperature=0, groq_api_key=GROQ_API_KEY
)

In [ ]:
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True
)

In [ ]:
query = "What does the document say about DELINEATING THE CLINICAL SYNDROME?"
response = qa_chain.invoke({"query":query})

In [ ]:
print(response)

{'query': 'What does the document say about DELINEATING THE CLINICAL SYNDROME?', 'result': 'The document does not provide a specific section on "DELINEATING THE CLINICAL SYNDROME". However, it does mention the importance of defining the clinical syndrome being evaluated in the following passage:\n\n"The starting point is a basic understanding of the pathophysiology of the respiratory tract and ways in which innate and acquired immune systems interact with microbial pathogens. The physician then defines the clinical syndrome being evaluated, the medical characteristics of the affected patient and context in which the infection was contracted."\n\nThis suggests that defining the clinical syndrome is an important step in the process of evaluating a patient with a respiratory tract infection.', 'source_documents': [Document(metadata={'author': 'Kurt D. Reed', 'creationDate': "D:20140828104522+05'30'", 'creator': 'Elsevier', 'file_path': '/content/drive/My Drive/Colab Notebooks/multidocqara

In [ ]:
print(response["result"])

The document does not provide a specific section on "DELINEATING THE CLINICAL SYNDROME". However, it does mention the importance of defining the clinical syndrome being evaluated in the following passage:

"The starting point is a basic understanding of the pathophysiology of the respiratory tract and ways in which innate and acquired immune systems interact with microbial pathogens. The physician then defines the clinical syndrome being evaluated, the medical characteristics of the affected patient and context in which the infection was contracted."

This suggests that defining the clinical syndrome is an important step in the process of evaluating a patient with a respiratory tract infection.


In [ ]:
query = "What does the document say about Receptor-binding domain?"
response = qa_chain.invoke({"query":query})
print(response["result"])

The document discusses the Receptor-binding domain (RBD) in the context of SARS-CoV-2 and neutralizing antibodies. Here are the key points mentioned:

1. **Location and structure**: The RBD is located on the S1 subunit of the spike protein and is responsible for binding to the ACE2 receptor on host cells.
2. **ACE2 binding site**: The RBD has a specific binding site for ACE2, which is essential for viral entry into host cells.
3. **Mutations in RBD**: The document mentions that many mutations in the RBD have occurred in various SARS-CoV-2 variants, including the Omicron sublineage. These mutations can affect the binding affinity of the RBD for ACE2 and the potency of neutralizing antibodies.
4. **Neutralizing antibodies targeting RBD**: The document highlights the importance of neutralizing antibodies that target the RBD, particularly those that bind near the neck and shoulders of the RBD, as these can block ACE2 binding and have strong neutralizing abilities.
5. **Examples of RBD-targ